In [1]:
import cv2
import numpy as np
import pandas as pd

import pytesseract
from pytesseract import Output

import re

import skimage.filters as filters
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import PIL.Image

import os, random, pathlib, warnings, itertools, math, imutils
warnings.filterwarnings("ignore")

import import_ipynb
#from ipynb.fs.full.data_exploring import orig1

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21))

In [4]:
img = cv2.imread(r'C:\Users\LENOVO\Desktop\OCR_extraction\Image_extraction\Testing_Data\Front\gk.jpg')
orig1 = img

In [5]:
mask = np.zeros(orig1.shape, dtype=np.uint8)

gray = cv2.cvtColor(orig1, cv2.COLOR_BGR2GRAY)

smooth = cv2.GaussianBlur(gray, (95,95), 0)

rgb = cv2.cvtColor(orig1, cv2.COLOR_BGR2RGB)

rgb1 = Image.frombytes('RGB', orig1.shape[:2], orig1, 'raw', 'BGR', 0, 0)

division = cv2.divide(gray, smooth, scale=255)

sharp = filters.unsharp_mask(division, radius=1.5, amount=1.5, channel_axis=False, preserve_range=False)

sharp = (255*sharp).clip(0,255).astype(np.uint8)

#thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [6]:
smooth = cv2.GaussianBlur(gray, (3, 3), 0)#blurring(3x3) to reduce noise and for smoothening

blackhat = cv2.morphologyEx(smooth, cv2.MORPH_BLACKHAT, sqKernel)#used to enhance dark objects on a light background

gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)#GRADIENT IN X DIRECTION 

gradX = np.absolute(gradX)#CHANGING ARRAY TO ABSOLUTE VALUES

(minVal, maxVal) = (np.min(gradX), np.max(gradX))#finding min value and max value in the matrix

gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")

gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)#gives outline

thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)

thresh = cv2.erode(thresh, None, iterations=4)

In [7]:
#cv2.imshow('thresh', thresh)
#cv2.imshow('mask', mask)

In [ ]:
plt.imshow(thresh,'gray')
# Filter for ROI using contour area and aspect ratio
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.05 * peri, True)
    x,y,w,h = cv2.boundingRect(approx)
    aspect_ratio = w / float(h)
    if area > 2000 and aspect_ratio > .5:
        mask[y:y+h, x:x+w] = orig1[y:y+h, x:x+w]
        cv2.rectangle(orig1, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [8]:
p = int(orig1.shape[1] * 0.05)
#print(p)
thresh[:, 0:p] = 0
thresh[:, orig1.shape[1] - p:] = 0
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#cv2_imshow(cnts)
#print(cnts)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    crWidth = w / float(gray.shape[1])
    pX = int((x + w) * 0.03)
    pY = int((y + h) * 0.03)
    (x, y) = (x - pX, y - pY)
    (w, h) = (w + (pX * 2), h + (pY * 2))

    roi = orig1[y:y + h, x:x + w].copy()
    cv2.rectangle(orig1, (x, y), (x + w, y + h), (0, 255, 0), 2)
    #filename="{}.jpg".format(os.getpid())
    #cv2.imwrite(filename,roi)

In [ ]:
plt.imshow(orig1)

In [10]:
thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [11]:
tessdata_dir_config = r'C:\Program Files\Tesseract-OCR\tesseract.exe\tessdata' 
tessdata_dir_config1 = r'C:\Program Files\Tesseract-OCR\tesseract.exe\tessdata_fast' 
custom_config0 = r'--oem 3 --psm 6'
custom_config1 = r'--oem 3 --psm 11'
custom_config2 = r'--oem 3 --psm 8'
custom_config3 = r'--oem 3 --psm 6 outputbase digits'

In [12]:
# Perfrom OCR with Pytesseract
#text1 = pytesseract.image_to_data(thresh,output_type='data.frame',lang='fra', config=custom_config1)
#text2 = pytesseract.image_to_string(thresh, lang='fra', config=custom_config1)  #mask or thresh 
text2 = pytesseract.image_to_string(thresh, lang='fra', config=tessdata_dir_config1)  #mask or thresh 

In [ ]:
print(text2)

In [14]:
#nums = re.findall(r'^[A-Z]{3,}[0-9]{6,}.', text2)
nums = re.findall(r'[A-Z][0-9][0-9][0-9][0-9][0-9][0-9].|[A-Z][0-9][0-9][0-9][0-9][0-9]', text2)
dates = re.findall(r'^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$', text2) 

NAME1 = r"[A-Z][a-z]+,?\s+"
MIDDLE_I = r"(?:[A-Z][a-z]*\.?\s*)?"
NAME2 = r"[A-Z][a-z]+"
names = re.findall(NAME1 + MIDDLE_I + NAME2, text2)

In [ ]:
print("NAME")
for name in names:
	print(name.strip())

In [ ]:
print("ID NUMBERS")
for num in nums:
	print(num.strip())

In [ ]:
print("BIRTH DATE")
for date in dates:
	print(dates.strip())